In [1]:
# Importing dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
from datetime import datetime
from citipy import citipy
from config import weather_api_key

In [2]:
# Create a set of 2000 random latitude and longitude combinations.
lats = np.random.uniform(low = -90.000, high = 90.000, size = 2000)
longs = np.random.uniform(low = -180.000, high = 180.000, size = 2000)
lats_longs = zip(lats, longs)
coordinates = list(lats_longs)

In [3]:
# Pull the nearest city using citipy.
cities = []
# Identify the city for each lats and longs coordinates.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, add to list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

727

In [4]:
# Creating url for Weather map API call request.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create a list to hold weather data.
city_data = []

# Print to indicate Data Retrieval Delete Later.
print("Beginning Data Retrieval, my Lord.")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval, my Lord.
-----------------------------


In [6]:
# Some of the print commands have been changed to messages. Used to make sure the operation works, but removed to save space on github.

In [7]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"My Lord, Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API call for each city.
    try:
        # Review the JSON then grab data.
        city_weather = requests.get(city_url).json()
        
        # Spread the required data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_wdesc = city_weather["weather"][0]["description"]
        try:
            city_rain = city_weather["rain"]
            try:
                city_rain_value = city_rain["3h"]
            except KeyError:
                city_rain_value = city_rain["1h"]
        except KeyError:
            city_rain_value = 0
        try:
            city_snow = city_weather["snow"]
            try:
                city_snow_value = city_snow["3h"]
            except KeyError:
                city_snow_value = city_snow["1h"]
        except KeyError:
            city_snow_value = 0
        
        # Append the city info into city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Latitudes": city_lat,
                          "Longitudes": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_wdesc,
                          "Rain (inches)": city_rain_value,
                          "Snow (inches)": city_snow_value})

    # If an error occurs, continue by skipping the city.
    except:
        #print("City cannot be found. Skipping...for now.")
        pass

    # Message for Data Retrieval being completed.
    #print("   ---   ---   ---   ---   ---   ---   ---   ")
    #print("     Data Retrieval Completed, my Lord.      ")
    #print("=============================================")
    

In [8]:
# Convert the array of dict. to Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Latitudes,Longitudes,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Hilo,US,19.7297,-155.0900,76.73,91,90,17.27,heavy intensity rain,5.06,0.0
1,Ostrovnoy,RU,68.0531,39.5131,9.59,73,83,23.38,broken clouds,0.00,0.0
2,Qaqortoq,GL,60.7167,-46.0333,25.30,68,100,23.62,overcast clouds,0.00,0.0
3,Rikitea,PF,-23.1203,-134.9692,76.50,74,99,8.61,overcast clouds,0.00,0.0
4,Hermanus,ZA,-34.4187,19.2345,60.91,94,100,10.36,light rain,0.52,0.0


In [9]:
# Create output file as csv.
output_data_file = "WeatherPy_database.csv"

# Convert the City_Data into a csv.
city_data_df.to_csv(output_data_file, index_label="City_ID")